In [1]:
import altair as alt
from altair.expr import datum, if_
import datetime
import json
import numpy as np
import pandas as pd
import psycopg2
pd.set_option("display.max_columns", 25)
pd.set_option("display.max_rows", 150)
pd.options.display.float_format = "{:,.2f}".format

Connect to the database.

In [2]:
with open("config.json") as f:
    conf = json.load(f)
conn = psycopg2.connect(
    dbname=conf["database"],
    user=conf["user"],
    host=conf["host"],
    password=conf["password"]
)
conn.autocommit = True

## Which families and corporations gave at least $50,000 to conservative and GOP-aligned groups between Nov. 2 and Dec. 31, 2017?

Return contributions by donor and committee.

In [3]:
post_bill_donors_committees_17 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_donors_committees_17 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2017-11-02'
     AND date <= '2017-12-31'
     AND CYCLE = '2018'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2018'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE post_bill_donors_committees_17 TO redash_default;


SELECT *
FROM post_bill_donors_committees_17;""", con=conn)
post_bill_donors_committees_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490 entries, 0 to 489
Data columns (total 6 columns):
match_id         490 non-null object
organizations    490 non-null object
contributors     490 non-null object
total            490 non-null float64
cmte_id          490 non-null object
committee        490 non-null object
dtypes: float64(1), object(5)
memory usage: 23.0+ KB


Group by donor.

In [4]:
post_bill_donors_17 = post_bill_donors_committees_17.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
post_bill_donors_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 4 columns):
match_id         157 non-null object
organizations    157 non-null object
contributors     157 non-null object
total            157 non-null float64
dtypes: float64(1), object(3)
memory usage: 5.0+ KB


## How much did they give?

In [5]:
post_bill_donors_17["total"].sum()

35588563.0

## Had these megadonors given previously in 2017?

Return contributions by donor and committee.

In [6]:
pre_bill_donors_committees_17 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_donors_committees_17 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2017-01-01'
     AND date < '2017-11-02'
     AND CYCLE = '2018'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2018'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE pre_bill_donors_committees_17 TO redash_default;


SELECT *
FROM pre_bill_donors_committees_17;""", con=conn)
pre_bill_donors_committees_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 831 entries, 0 to 830
Data columns (total 6 columns):
match_id         831 non-null object
organizations    831 non-null object
contributors     831 non-null object
total            831 non-null float64
cmte_id          831 non-null object
committee        831 non-null object
dtypes: float64(1), object(5)
memory usage: 39.0+ KB


Group by donor.

In [7]:
pre_bill_donors_17 = pre_bill_donors_committees_17.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 4 columns):
match_id         114 non-null object
organizations    114 non-null object
contributors     114 non-null object
total            114 non-null float64
dtypes: float64(1), object(3)
memory usage: 3.6+ KB


## How much did they give?

In [8]:
pre_bill_donors_17["total"].sum()

58732582.0

## How much did these donors give in the period leading up to the tax bill's introduction and in the period after its introduction?

In [9]:
donors_17 = post_bill_donors_17.merge(pre_bill_donors_17, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_17.drop(["organizations_pre_bill", "contributors_pre_bill"], axis=1, inplace=True)
donors_17.rename(columns={"organizations_post_bill": "organizations", "contributors_post_bill": "contributors"}, inplace=True)
donors_17["total_pre_bill"].fillna(0, inplace=True)
donors_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157 entries, 0 to 156
Data columns (total 5 columns):
match_id           157 non-null object
organizations      157 non-null object
contributors       157 non-null object
total_post_bill    157 non-null float64
total_pre_bill     157 non-null float64
dtypes: float64(2), object(3)
memory usage: 7.4+ KB


In [10]:
donors_17["pct_post_bill"] = donors_17["total_post_bill"] / (donors_17["total_pre_bill"] + donors_17["total_post_bill"])
donors_17["pct_pre_bill"] = donors_17["total_pre_bill"] / (donors_17["total_pre_bill"] + donors_17["total_post_bill"])
donors_17["change"] = donors_17["total_post_bill"] - donors_17["total_pre_bill"]
donors_17["pct_change"] = (donors_17["total_post_bill"] - donors_17["total_pre_bill"]) / donors_17["total_pre_bill"].abs()
donors_17.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations,contributors,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
51,U0000003690,"[U-Line Corp, Uline Inc]","[UIHLEIN, ELIZABETH, UIHLEIN, ELIZABETH MRS, U...","4,557,300.00","12,051,400.00",0.27,0.73,"-7,494,100.00",-0.62
2,American Action Network,[American Action Network],[AMERICAN ACTION NETWORK],"3,747,520.00","9,142,525.00",0.29,0.71,"-5,395,005.00",-0.59
59,U0000004054,"[Ghpalmer Assoc, GH Palmer Assoc]","[PALMER, GEOFF, PALMER, GEOFFREY H]","1,955,200.00","546,272.00",0.78,0.22,"1,408,928.00",2.58
9,Hillwood Development,[Hillwood Development],[HILLWOOD DEVELOPMENT COMPANY LLC],"1,500,000.00","500,000.00",0.75,0.25,"1,000,000.00",2.00
66,U0000004552,"[Cinemark Holdings, Cinemark USA]","[MITCHELL, LEE, MITCHELL, LEE ROY, MITCHELL, T...","1,007,400.00","131,299.00",0.88,0.12,"876,101.00",6.67


## Had these megadonors given between Nov. 2 and Dec. 31, 2013?

Return contributions by donor and committee.

In [11]:
post_bill_donors_committees_13 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_donors_committees_13 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2013-11-02'
     AND date <= '2013-12-31'
     AND CYCLE = '2014'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2014'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE post_bill_donors_committees_13 TO redash_default;


SELECT *
FROM post_bill_donors_committees_13;""", con=conn)
post_bill_donors_committees_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 6 columns):
match_id         188 non-null object
organizations    188 non-null object
contributors     188 non-null object
total            188 non-null float64
cmte_id          188 non-null object
committee        188 non-null object
dtypes: float64(1), object(5)
memory usage: 8.9+ KB


Group by donor.

In [12]:
post_bill_donors_13 = post_bill_donors_committees_13.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
post_bill_donors_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 4 columns):
match_id         61 non-null object
organizations    61 non-null object
contributors     61 non-null object
total            61 non-null float64
dtypes: float64(1), object(3)
memory usage: 2.0+ KB


## How much did they give?

In [13]:
post_bill_donors_13["total"].sum()

3816664.0

## Had these megadonors given previously in 2013?

Return contributions by donor and committee.

In [14]:
pre_bill_donors_committees_13 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_donors_committees_13 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2013-01-01'
     AND date < '2013-11-02'
     AND CYCLE = '2014'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2014'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE pre_bill_donors_committees_13 TO redash_default;


SELECT *
FROM pre_bill_donors_committees_13;""", con=conn)
pre_bill_donors_committees_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 6 columns):
match_id         518 non-null object
organizations    518 non-null object
contributors     518 non-null object
total            518 non-null float64
cmte_id          518 non-null object
committee        518 non-null object
dtypes: float64(1), object(5)
memory usage: 24.4+ KB


Group by donor.

In [15]:
pre_bill_donors_13 = pre_bill_donors_committees_13.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 4 columns):
match_id         104 non-null object
organizations    104 non-null object
contributors     104 non-null object
total            104 non-null float64
dtypes: float64(1), object(3)
memory usage: 3.3+ KB


## How much did they give?

In [16]:
pre_bill_donors_13["total"].sum()

6519675.0

## How much did these donors give in each period?

In [17]:
donors_13 = post_bill_donors_13.merge(pre_bill_donors_13, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_13["total_pre_bill"].fillna(0, inplace=True)
donors_13["total_post_bill"].fillna(0, inplace=True)
donors_13.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108 entries, 0 to 107
Data columns (total 7 columns):
match_id                   108 non-null object
organizations_post_bill    61 non-null object
contributors_post_bill     61 non-null object
total_post_bill            108 non-null float64
organizations_pre_bill     104 non-null object
contributors_pre_bill      104 non-null object
total_pre_bill             108 non-null float64
dtypes: float64(2), object(5)
memory usage: 6.8+ KB


In [18]:
donors_13["pct_post_bill"] = donors_13["total_post_bill"] / (donors_13["total_pre_bill"] + donors_13["total_post_bill"])
donors_13["pct_pre_bill"] = donors_13["total_pre_bill"] / (donors_13["total_pre_bill"] + donors_13["total_post_bill"])
donors_13["change"] = donors_13["total_post_bill"] - donors_13["total_pre_bill"]
donors_13["pct_change"] = (donors_13["total_post_bill"] - donors_13["total_pre_bill"]) / donors_13["total_pre_bill"].abs()
donors_13 = donors_13[["match_id", "organizations_post_bill", "contributors_post_bill",
                       "organizations_pre_bill", "contributors_pre_bill", "total_post_bill",
                       "total_pre_bill", "pct_post_bill", "pct_pre_bill", "change", "pct_change"]]
donors_13.head()

,match_id,organizations_post_bill,contributors_post_bill,organizations_pre_bill,contributors_pre_bill,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
0,American Action Network,[American Action Network],[AMERICAN ACTION NETWORK],[American Action Network],[AMERICAN ACTION NETWORK],"109,714.00","81,237.00",0.57,0.43,"28,477.00",0.35
1,Anschutz Corp,[Anschutz Corp],[THE ANSCHUTZ CORPORATION],NaN,NaN,"5,000.00",0.00,1.00,0.00,"5,000.00",inf
2,Republican Governors Assn,[Republican Governors Assn],[REPUBLICAN GOVERNORS ASSOCIATION],NaN,NaN,"1,000,000.00",0.00,1.00,0.00,"1,000,000.00",inf
3,Shakopee Mdewakanton Sioux Community,[Shakopee Mdewakanton Sioux Community],"[MDEWAKANTON SIOUX COMMUNITY, SHAKOPEE,,,, SHA...",[Shakopee Mdewakanton Sioux Community],"[MDEWAKANTON SIOUX COMMUNITY, SHAKOPEE,,,, SHA...","68,900.00","71,100.00",0.49,0.51,"-2,200.00",-0.03
4,U0000000003,"[Alticor Inc, Amway]","[DEVOS, RICHARD M, DEVOS, RICHARD M MR SR]","[Alticor Inc, Amway]","[DEVOS, RICHARD M, DEVOS, RICHARD M MR SR]","76,200.00","19,200.00",0.80,0.20,"57,000.00",2.97


## And did their giving patterns change between the cycles?

In [19]:
donors = donors_17.merge(donors_13, how="outer", on="match_id", suffixes=["_17", "_13"])
donors.drop(["change_17", "pct_change_17", "organizations_post_bill", "contributors_post_bill",
             "organizations_pre_bill", "contributors_pre_bill", "change_13", "pct_change_13"],
            axis=1, inplace=True)
donors.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"})
donors = donors[["match_id", "contributors", "organizations", "total_pre_bill_13", "total_post_bill_13",
                 "pct_pre_bill_13", "pct_post_bill_13", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157 entries, 0 to 156
Data columns (total 11 columns):
match_id              157 non-null object
contributors          157 non-null object
organizations         157 non-null object
total_pre_bill_13     108 non-null float64
total_post_bill_13    108 non-null float64
pct_pre_bill_13       108 non-null float64
pct_post_bill_13      108 non-null float64
total_pre_bill_17     157 non-null float64
total_post_bill_17    157 non-null float64
pct_pre_bill_17       157 non-null float64
pct_post_bill_17      157 non-null float64
dtypes: float64(8), object(3)
memory usage: 14.7+ KB


In [20]:
donors["giving_change"] = np.where((donors["pct_post_bill_17"] > donors["pct_post_bill_13"]) | (donors["pct_pre_bill_13"].isnull()), "Increased",
                                  np.where(donors["pct_post_bill_17"] < donors["pct_post_bill_13"], "Decreased",
                                           np.where(donors["pct_post_bill_17"] == donors["pct_post_bill_13"], "Stayed the same",
                                                   "Other")))
donors.head(1)

,match_id,contributors,organizations,total_pre_bill_13,total_post_bill_13,pct_pre_bill_13,pct_post_bill_13,total_pre_bill_17,total_post_bill_17,pct_pre_bill_17,pct_post_bill_17,giving_change
0,Air Line Pilots Assn,[AIR LINE PILOTS ASSOC INT'L POLITICAL ACTIO...,[Air Line Pilots Assn],nan,nan,nan,nan,"100,000.00","150,000.00",0.40,0.60,Increased


## What proportion of donors increased the share of their annual giving that fell in the last two months of 2017 as compared with the same period in 2013?

In [21]:
donors[donors["giving_change"] == "Increased"].count()

match_id              131
contributors          131
organizations         131
total_pre_bill_13      82
total_post_bill_13     82
pct_pre_bill_13        82
pct_post_bill_13       82
total_pre_bill_17     131
total_post_bill_17    131
pct_pre_bill_17       131
pct_post_bill_17      131
giving_change         131
dtype: int64

## And by how much did these individuals' giving increase?

In [22]:
donors[donors["giving_change"] == "Increased"]["total_post_bill_17"].sum()

22986493.0

In [23]:
donors[donors["giving_change"] == "Increased"]["total_post_bill_13"].sum()

1155975.0

In [24]:
donors[donors["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors[donors["giving_change"] == "Increased"]["total_post_bill_13"].sum()

21830518.0

# What does this giving look like on a contribution-by-contribution level?

During the period after the tax bill was introduced?

In [25]:
post_bill_contributions_17 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_contributions_17 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2017-11-02'
     AND date <= '2017-12-31'
     AND CYCLE = '2018'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2018'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE post_bill_contributions_17 TO redash_default;


SELECT *
FROM post_bill_contributions_17;""", con=conn)
post_bill_contributions_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 8 columns):
fectransid      895 non-null object
match_id        895 non-null object
organization    895 non-null object
contributor     895 non-null object
date            895 non-null object
sum             895 non-null float64
cmte_id         895 non-null object
committee       895 non-null object
dtypes: float64(1), object(7)
memory usage: 56.0+ KB


During the period before the tax bill was introduced?

In [26]:
pre_bill_contributions_17 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_contributions_17 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2017-01-01'
     AND date < '2017-11-02'
     AND CYCLE = '2018'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2018'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE pre_bill_contributions_17 TO redash_default;


SELECT *
FROM pre_bill_contributions_17;""", con=conn)
pre_bill_contributions_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1654 entries, 0 to 1653
Data columns (total 8 columns):
fectransid      1654 non-null object
match_id        1654 non-null object
organization    1654 non-null object
contributor     1654 non-null object
date            1654 non-null object
sum             1654 non-null float64
cmte_id         1654 non-null object
committee       1654 non-null object
dtypes: float64(1), object(7)
memory usage: 103.5+ KB


What about during the equivalent post-bill introduction period in 2013?

In [27]:
post_bill_contributions_13 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_contributions_13 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2013-11-02'
     AND date <= '2013-12-31'
     AND CYCLE = '2014'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2014'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE post_bill_contributions_13 TO redash_default;


SELECT *
FROM post_bill_contributions_13;""", con=conn)
post_bill_contributions_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 8 columns):
fectransid      303 non-null object
match_id        303 non-null object
organization    303 non-null object
contributor     303 non-null object
date            303 non-null object
sum             303 non-null float64
cmte_id         303 non-null object
committee       303 non-null object
dtypes: float64(1), object(7)
memory usage: 19.0+ KB


In [28]:
pre_bill_contributions_13 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_contributions_13 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2013-01-01'
     AND date < '2013-11-02'
     AND CYCLE = '2014'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2014'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE pre_bill_contributions_13 TO redash_default;


SELECT *
FROM pre_bill_contributions_13;""", con=conn)
pre_bill_contributions_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 893 entries, 0 to 892
Data columns (total 8 columns):
fectransid      893 non-null object
match_id        893 non-null object
organization    893 non-null object
contributor     893 non-null object
date            893 non-null object
sum             893 non-null float64
cmte_id         893 non-null object
committee       893 non-null object
dtypes: float64(1), object(7)
memory usage: 55.9+ KB


## Export data to Excel.

In [29]:
writer = pd.ExcelWriter("data/megadonors.xlsx")
donors.to_excel(writer, "donors", startcol=0, index=False)
post_bill_donors_committees_17.to_excel(writer, "post_bill_donors_committees_17", startcol=0, index=False)
pre_bill_donors_committees_17.to_excel(writer, "pre_bill_donors_committees_17", startcol=0, index=False)
post_bill_donors_committees_13.to_excel(writer, "post_bill_donors_committees_13", startcol=0, index=False)
pre_bill_donors_committees_13.to_excel(writer, "pre_bill_donors_committees_13", startcol=0, index=False)
post_bill_contributions_17.to_excel(writer, "post_bill_contributions_17", startcol=0, index=False)
pre_bill_contributions_17.to_excel(writer, "pre_bill_contributions_17", startcol=0, index=False)
post_bill_contributions_13.to_excel(writer, "post_bill_contributions_13", startcol=0, index=False)
pre_bill_contributions_13.to_excel(writer, "pre_bill_contributions_13", startcol=0, index=False)
writer.save()